In [19]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, IntSlider


# Generate synthetic data: y = x^2 + noise
def generate_data(n):
    x = np.random.uniform(-1, 1, n)
    y = x**2 + np.random.normal(0, 0.1, n)
    return x, y

# Polynomial regression
def poly_regression(x, y, degree):
    poly = PolynomialFeatures(degree)
    X_poly = poly.fit_transform(x.reshape(-1, 1))
    model = LinearRegression().fit(X_poly, y)
    return model, poly

# Compute bias and variance
def compute_bias_variance(degree, n, trials=10):
    test_x, test_y = generate_data(100)
    predictions = []
    
    for _ in range(trials):
        train_x, train_y = generate_data(n)
        model, poly = poly_regression(train_x, train_y, degree)
        X_test_poly = poly.transform(test_x.reshape(-1, 1))
        preds = model.predict(X_test_poly)
        predictions.append(preds)
    
    predictions = np.array(predictions)
    mean_preds = np.mean(predictions, axis=0)
    bias_squared = np.mean((mean_preds - test_y)**2)
    variance = np.mean(np.var(predictions, axis=0))
    return bias_squared, variance, bias_squared + variance

# Store history for trend plot
history = {'bias_squared': [], 'variance': [], 'total_error': [], 'degrees': [], 'n': None}

# Interactive plot
def plot_bias_variance(degree=1, n=20):
    global history
    
    # Compute for selected degree
    bias_squared, variance, total_error = compute_bias_variance(degree, n)
    
    # Update history
    if n != history['n']:
        history = {'bias_squared': [], 'variance': [], 'total_error': [], 'degrees': [], 'n': n}
    if degree not in history['degrees']:
        history['degrees'].append(degree)
        history['bias_squared'].append(bias_squared)
        history['variance'].append(variance)
        history['total_error'].append(total_error)
    
    # Sort history
    sorted_idx = np.argsort(history['degrees'])
    sorted_degrees = np.array(history['degrees'])[sorted_idx]
    sorted_bias = np.array(history['bias_squared'])[sorted_idx]
    sorted_variance = np.array(history['variance'])[sorted_idx]
    sorted_total = np.array(history['total_error'])[sorted_idx]
    
    # Generate data for plotting points and polynomials
    train_x, train_y = generate_data(n)
    x_smooth = np.linspace(-1, 1, 100)  # For smooth polynomial curves
    
    # Create three subplots
    fig = plt.figure(figsize=(15, 5))
    
    # Plot 1: Data points and polynomial fits
    ax1 = fig.add_subplot(131)
    ax1.scatter(train_x, train_y, color='black', label='Data', s=50)
    colors = ['blue', 'red', 'green', 'purple', 'orange']
    for d in range(1, degree + 1):
        model, poly = poly_regression(train_x, train_y, d)
        X_smooth_poly = poly.transform(x_smooth.reshape(-1, 1))
        y_smooth = model.predict(X_smooth_poly)
        ax1.plot(x_smooth, y_smooth, label=f'Degree {d}', color=colors[d-1 % len(colors)])
    ax1.set_title(f'Data and Polynomial Fits (n={n})')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.legend()
    ax1.grid(True)
    
    # Plot 2: Bar plot for current degree
    ax2 = fig.add_subplot(132)
    ax2.bar(['Bias²', 'Variance', 'Total Error'], [bias_squared, variance, total_error], 
            color=['blue', 'red', 'green'])
    ax2.set_title(f'Errors for Degree {degree}')
    ax2.set_ylabel('Error')
    ax2.grid(True, axis='y')
    
    # Plot 3: Line plot for history
    ax3 = fig.add_subplot(133)
    ax3.plot(sorted_degrees, sorted_bias, label='Bias²', color='blue')
    ax3.plot(sorted_degrees, sorted_variance, label='Variance', color='red')
    ax3.plot(sorted_degrees, sorted_total, label='Total Error', color='green')
    ax3.set_xlabel('Polynomial Degree')
    ax3.set_ylabel('Error')
    ax3.set_title('Error Trends')
    ax3.legend()
    ax3.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create interactive sliders
interact(
    plot_bias_variance,
    degree=IntSlider(min=1, max=5, step=1, value=1, description='Degree'),
    n=IntSlider(min=10, max=50, step=5, value=20, description='Data Size')
)

interactive(children=(IntSlider(value=1, description='Degree', max=5, min=1), IntSlider(value=20, description=…

<function __main__.plot_bias_variance(degree=1, n=20)>